In [27]:
from tensorflow_core.keras.models import load_model
from tensorflow_core.keras.preprocessing import image
import numpy as np
import cv2
import wx
import wx.lib.filebrowsebutton
from PIL import Image

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# load the model we saved
model = load_model('covid2model')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [23]:
# predicting images
# import urllib.request
# req = urllib.request.urlopen('http://answers.opencv.org/upfiles/logo_2.png')
# img = image.load_img(req, target_size=(img_width, img_height))
(img_width, img_height) = (224, 224)
img = image.load_img('dataset/covid/1-s2.0-S0929664620300449-gr2_lrg-a.jpg', target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images/255, batch_size=10)
print(np.argmax(classes, axis=1))
print(classes)

[0]
[[0.8219949  0.17800507]]


In [12]:
(img_width, img_height) = (224, 224)
img = image.load_img('dataset/normal/IM-0466-0001.jpeg', target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images/255, batch_size=10)
print(np.argmax(classes, axis=1))
print(classes)

[1]
[[0.2824068 0.7175932]]


In [13]:
classes[0][1] 

0.7175932

In [24]:
def is_affected(path, img_height, img_width):
    img = image.load_img(path, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    
    images = np.vstack([x])
    classes = model.predict(images/255, batch_size = 10)
    if np.argmax(classes, axis=1) == 0:
        out = "Covid Positive with " + str(round(classes[0][0] * 100)) + "% Accuracy"
        return out
    else:
        out = "Covid Negative with " + str(round(classes[0][1] * 100)) + "% Accuracy"
        return out

In [25]:
class MyFrame(wx.Frame):    
    def on_show(self, event):
        path = str(self.fbb.GetValue())
        img  = Image.open(path)
        img.show()
        
    def on_perform(self, event):
        path = str(self.fbb.GetValue())
        output = is_affected(path, self.img_height, self.img_width)
        wx.MessageBox(str(output), "Result" ,wx.OK | wx.ICON_INFORMATION)
            
    def __init__(self):
        super().__init__(parent=None, title='Covid-19 Detection', size = (600,480))
        self.img_width, self.img_height = 224, 224
        self.panel = wx.Panel(self)        
        self.Bind(wx.EVT_CLOSE, self.OnCloseWindow)
        my_sizer = wx.BoxSizer(wx.VERTICAL)    
        
        self.fbb = wx.lib.filebrowsebutton.FileBrowseButton(self.panel,
            labelText="Select Image to Predict (jpg):", fileMask="*.jpg; *.jpeg")
        my_sizer.Add(self.fbb, 0, wx.ALL | wx.CENTER, 10)
        
        
        btn2 = wx.Button(self.panel, label='Show Image')
        btn2.Bind(wx.EVT_BUTTON, self.on_show)
        my_sizer.Add(btn2, 0, wx.ALL | wx.CENTER, 5) 
        
        btn3 = wx.Button(self.panel, label='Perform Test')
        btn3.Bind(wx.EVT_BUTTON, self.on_perform)
        my_sizer.Add(btn3, 0, wx.ALL | wx.CENTER, 5) 
        
        self.label_output = wx.StaticText(self.panel,-1,style = wx.ALIGN_CENTER)
        my_sizer.Add(self.label_output, 0, wx.ALL | wx.EXPAND, 10)
        
        self.panel.SetSizer(my_sizer)        
        self.Show()
        
    def OnCloseWindow(self, e):
        self.Destroy()

In [26]:
app = wx.App()
frame = MyFrame()
app.MainLoop()

0

In [20]:
# Run this Cell when "wx.App object must be created first!" Error Appears
del app